<a href="https://colab.research.google.com/github/vishnupriya-ms/My-Projects/blob/main/NextWordGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.optimizers import RMSprop

In [ ]:
df=pd.read_csv("/content/email_spam.csv")
df_text=list(df.text.values)
text=" ".join(df_text) # joins all the text values from the df_text list into a single string, separated by a space.

In [ ]:
#Tokenizing the text to create a sequence of words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index) + 1  #calculates the total number of unique words in the vocabulary

In [ ]:
#Creating input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences. The texts_to_sequences() method converts a list of texts into a list of sequences of integers
input_sequences=[]
for line in text.split('\n'):
  token_list=tokenizer.texts_to_sequences([line])[0] #tokenizes the text into a sequence of integers using the previously initialized tokenizer.
  for i in range(1,len(token_list)):
    n_gram_sequence= token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
#Input sequences are padded to ensure that all sequences have the same length.
max_seq_len=max([len(seq) for seq in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_seq_len,padding='pre'))

In [ ]:
X=input_sequences[:,:-1]  #>>>>>except the last column
y=input_sequences[:,-1]   #>>>>>Last column

In [ ]:
#Converting each target vector as a binary vector
y=np.array(tf.keras.utils.to_categorical(y,num_classes=total_words))

In [ ]:
model=Sequential()  #A sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
model.add(Embedding(total_words,100,input_length=max_seq_len-1))
model.add(LSTM(150))
model.add(Dense(total_words,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 169, 100)          296900    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 2969)              448319    
                                                                 
Total params: 895819 (3.42 MB)
Trainable params: 895819 (3.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(learning_rate=0.01),metrics=["accuracy"])
model.fit(X,y,batch_size=128, epochs=30, shuffle=True)  #batch size==>Specifies the number of samples per gradient update.

Epoch 1/30
85/85 [==============================] - 68s 731ms/step - loss: 7.0534 - accuracy: 0.0294
Epoch 2/30
85/85 [==============================] - 62s 736ms/step - loss: 6.7186 - accuracy: 0.0386
Epoch 3/30
85/85 [==============================] - 63s 738ms/step - loss: 6.4411 - accuracy: 0.0528
Epoch 4/30
85/85 [==============================] - 63s 740ms/step - loss: 6.0618 - accuracy: 0.0784
Epoch 5/30
85/85 [==============================] - 62s 734ms/step - loss: 5.6076 - accuracy: 0.1119
Epoch 6/30
85/85 [==============================] - 63s 736ms/step - loss: 5.1262 - accuracy: 0.1484
Epoch 7/30
85/85 [==============================] - 62s 731ms/step - loss: 4.6312 - accuracy: 0.1918
Epoch 8/30
85/85 [==============================] - 63s 742ms/step - loss: 4.1418 - accuracy: 0.2462
Epoch 9/30
85/85 [==============================] - 64s 750ms/step - loss: 3.6589 - accuracy: 0.3037
Epoch 10/30
85/85 [==============================] - 64s 751ms/step - loss: 3.1925 - accura

In [ ]:
# Function to generate the next word
def generate_next_word(model, tokenizer, max_seq_len, seed_text, num_next_words):
    for _ in range(num_next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Seed text for prediction
seed_text = "You are among "

# Number of words to predict
num_next_words = 5

# Generate next word
next_word = generate_next_word(model, tokenizer, max_seq_len, seed_text, num_next_words)
print("Predicted next word:", next_word)


Predicted next word: You are among  excited to do something stupid


In [ ]:
# Function to generate the next word
def generate_next_word(model, tokenizer, max_seq_len, seed_text, num_next_words):
    for _ in range(num_next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Seed text for prediction
seed_text = "Have you"

# Number of words to predict
num_next_words = 8

# Generate next word
next_word = generate_next_word(model, tokenizer, max_seq_len, seed_text, num_next_words)
print("Predicted next word:", next_word)


Predicted next word: Have you been successful in winning projects and earning more


In [ ]:
4# Define the chatbot function
def chatbot():
    print("Welcome to the Chatbot! Type 'exit' to end the conversation.")

    # Main loop to receive input and generate responses
    while True:
        # Receive input from the user
        user_input = input("You: ")

        # Check for exit condition
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break

        # Generate response
        response = generate_next_word(model, tokenizer, max_seq_len, user_input, num_next_words=5)
        print("Chatbot:", response)

# Run the chatbot
chatbot()


Welcome to the Chatbot! Type 'exit' to end the conversation.
You: the name of
Chatbot: the name of your account is amazon and
You: exit
Chatbot: Goodbye!
